### 1. Load Environment Variables from .env 📥

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 2. Load Documents 📚

In [ ]:
from langchain.document_loaders import PyPDFLoader

documents = PyPDFLoader('documents/book_01.pdf').load()
documents

### 3. Preprocess Documents & Create Chunks 📝

In [3]:
from langchain.text_splitter import CharacterTextSplitter

chunks = CharacterTextSplitter(chunk_size= 1000, chunk_overlap=0).split_documents(documents)

### 4. Create Embeddings 📐

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

### 5. Initialize The Database 🗄️

In [7]:
from langchain.vectorstores import Chroma

database = Chroma.from_documents(chunks, embeddings)

### 6. Create The Retriever 🧲

In [8]:
retriever = database.as_retriever(search_type='similarity', search_kwargs={'k': 3})

### 7. Import The LLM Model 🤖

In [9]:
from langchain.llms.openai import OpenAI

llm = OpenAI()

### 8. Build Your Chain 📜

In [10]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
)

### 9. Start Testing 🧪

In [70]:
def ask(question):
    prompt = f"""
    You are an advanced language model designed to analyze and understand medical literature, specifically focused on chest pneumonia. Your goal is to provide comprehensive information on diagnoses and treatment related to chest pneumonia. If the information is available in the documents you have been trained on, please provide detailed and accurate responses. If the information is not present, respond with 'Not mentioned in the documents.'
    Ensure that your answers are based on credible medical sources and provide a clear distinction between known information and unknown information.
    Question: {question}
    """
    output = qa({'query' : prompt})
    result, source = output['result'], output['source_documents']

    print(f'Query :  {question}')
    print(f'Result:  {result}')
    if result.find('Not mentioned in the documents') < 0:
        print(f'From  :  {source[0].metadata["source"]}, page number: {source[0].metadata["page"] + 1}')

In [71]:
ask('What are the symptoms of pneumonia?')

Query :  What are the symptoms of pneumonia?
Result:   The symptoms of pneumonia include fever, cough, tachypnea, and rales on auscultation.
From  :  documents/book_01.pdf, page number: 3


In [72]:
ask("how to treat pneumonia?")

Query :  how to treat pneumonia?
Result:   Pneumonia can be treated with a variety of supportive treatments, including oxygen delivery methods, fluid therapy regimens, chest physiotherapy, steroid treatment, granulocyte colony stimulating factor treatment, surfactant application, vitamins A, C and D, zinc, protein and calorie supplements. Monitoring of oxygen levels and oxygen delivery guided by oxygen saturation levels can also reduce hospitalization. In children, nasal prongs and nasopharyngeal catheters have been found to be similarly effective and safe. Clinical signs may not reliably indicate hypoxemia, but vitamin C supplementation in critically ill patients has been associated with lower mortality and reduced respiratory symptom score in one trial. Steroid application has been associated with reduction of mortality and need for mechanical ventilation in people with pneumocystis jiroveci pneumonia, but there is insufficient evidence to support their use in other forms of pneumoni

In [73]:
ask("What is the most common cause of pneumonia?")

Query :  What is the most common cause of pneumonia?
Result:   The most common cause of pneumonia is an inflammatory-infectious process. However, other etiological causes of inflammation in the lung, such as aerobic gram-negative germs, viruses, fungi, and atypical pathogens, can also cause pneumonia.
From  :  documents/book_01.pdf, page number: 13


In [74]:
ask("what is the fatality rate of pneumonia?")

Query :  what is the fatality rate of pneumonia?
Result:   The American Thoracic Society estimates that the mortality rate of pneumonia is 6 to 12% in patients admitted to hospital with community-acquired pneumonia and over 50% in patients admitted to the intensive care unit.
From  :  documents/book_01.pdf, page number: 13


In [75]:
ask("what is the rate of tuberculosis?")

Query :  what is the rate of tuberculosis?
Result:   Not mentioned in the documents.
